<h1>Hub Selection Model</h1>
<br>
This script serves as the skeleton of <b>strategic network optimization</b>. Expected to be extended / modified with ease for general purpose logistics / supply chain transportation planning.
<br>
<br>
Consider set of nodes $n\in N$. For node pair $[n,n'], n \neq n'$ forming origin-destination pairs $od \in OD$ in a network. For each $od \in OD$, try to assign a routing path $\alpha^p_{od}$ (A,B,C....h) among all possible routing for the od through hubs $h \in H \in N$ such that total transportation cost $\sum_p v^p\alpha^p_od + \sum_h \beta_h f_h$ is minimized, subject to hub capacity $\sum_p s_p^{od} \leq \beta_h C_h \forall h, od$

In [201]:
from cvxpy import *
from itertools import combinations
from scipy.sparse import coo_matrix
import numpy as np
import time

<h2>Demo Data Preparation</h2>
<br>
<b>OD pairs - dictionary</b><br>
key:   tuple(from, to, time_allowed)<br>value: dictionary of od-pair characteristics. Weight used in demo as flow volume

In [5]:
od = {
    ('A','B',24): {'weight':300},
    ('A','B',48): {'weight':600}
}

<b>Hubs - dictionary</b><br>
key:   node name<br>
value: FC:  fixed operating cost; VC:  variable cost / flow unit; Cap: facility capacity


In [125]:
hubs = {
    'C': {'fc':40000,'vc':1.5,'cap':2000},
    'D': {'fc':20000,'vc':1.5,'cap':2000}
}

<b>Distance matrix - dictionary</b><br>
key:   tuple of OD pair<br>
value: distance

In [10]:
dist_matrix = {
    ('A','B'): 3000,
    ('B','C'): 600,
    ('A','C'): 550,
    ('A','D'): 1500,
    ('B','A'): 450,
    ('B','D'): 900,
    ('C','A'): 650,
    ('C','B'): 1100,
    ('C','D'): 540,
    ('D','A'): 690,
    ('D','B'): 800,
    ('D','C'): 440
}

<h2>Config</h2>
<ul>
    <li>max_hubs:  maximum number of hubs allowed to be used for OD routing</li>
    <li>vehicle_speed:  speed of vehicle used to deduct transportation time consumption</li>
    <li>transit_vc:  generalized transportation variable cost over the network</li>
    <li>facility_stay_time: additional travel time for each stop passed in OD</li>
</ul>

In [24]:
CONFIG = {
    'max_hubs':2,
    'vehicle_speed': 90,
    'transit_vc': 0.00066,
    'facility_stay_time': 1
}


<h3>Candidate routes combinations generation</h3>

In [185]:
def gen_ods_list(od, hubs, dist_matrix):
    ods = []
    for od_pair in od:
        for c in range(1,CONFIG['max_hubs']+1):
            for hub in combinations(hubs, c):
                p = [od_pair[0]] + list(hub) + [od_pair[1]]
                tt = find_transit_time(p, dist_matrix)
                if tt <= od_pair[2]:
                    ods.append({
                        'od': od_pair,
                        'weight': od[od_pair]['weight'],
                        'path': p,
                        'transit_time': tt,
                        'var_cost': find_var_cost(p, dist_matrix, hubs) * od[od_pair]['weight']
                    })
    return ods

ods = gen_ods_list(od, hubs, dist_matrix)

In [1]:
def find_transit_time(p, dist_matrix):
    """
        find transit time of path p from lookup table (dist_matrix)
        @params:
            p - ordered iterable (e.g. list) representing path travesal sequence
            dist_matrix - distance matrix lookup table
        @return:
            t - total transit + facility stay time
    """
    t = 0
    for i in range(len(p)-1):
        t += dist_matrix[p[i],p[i+1]] / CONFIG['vehicle_speed']
        if CONFIG['facility_stay_time'] is not None:
            t += CONFIG['facility_stay_time']
    return t
    
    
def find_var_cost(p, dist_matrix, hubs):
    """
        find variable cost of path p from distance matrix and hubs data
        @params:
            p - ordered iterable (e.g. list) representing path travesal sequence
            dist_matrix - distance matrix lookup table
            hubs - hubs dictionary storing all facility operating characteristics
        @return:
            c - total variable cost
    """
    c = 0
    for i in range(len(p)-1):
        c += dist_matrix[p[i],p[i+1]] * CONFIG['transit_vc']
        
    for i in range(1,len(p)-1):
        c += hubs[p[i]]['vc']
    return c
    

In [204]:
def solve(hubs, od, dm):
    """
        Solve hub selection and network routing problem with capacity and transit time constraint.
        Model characteristics:
            - Single transportation mode
            - Complete fulfillment
            - No split shipments
        
        This function will convert all dictionary into numpy matrices and construct a linear optimization model.
        For large matrix input / parameters, coo_matrix from scipy.sparse will be deployed. 
    
    """
    ods = gen_ods_list(od, hubs, dm)

    hub_2_idx = {}
    idx_2_hub = {}
    hub_idx = 0

    od_2_idx = {}
    idx_2_od = {}
    od_idx = 0

    for hub in hubs:
        hub_2_idx[hub] = hub_idx
        idx_2_hub[hub_idx] = hub
        hub_idx +=1

    for i in od:
        od_2_idx[i] = od_idx
        idx_2_od[od_idx] = i
        od_idx += 1

    # shape:  1 x ods
    vc = np.zeros(len(ods))

    # shape:  1 x hubs
    fc = np.zeros(len(hubs))

    # shape:  hub * ods  (row)
    cap_row = []
    cap_col = []
    cap_val = []

    # shape: 1 * hub
    cap_lim = np.zeros(len(hubs))

    # shape:  od_pair * ods  (row)
    demand_row = []
    demand_col = []
    demand_val = []

    # shape:  1 * od
    demand_lim = np.zeros(len(od))
    
    
    for i in range(0,len(ods)):
        vc[i] = ods[i]['var_cost']

    for h in hubs:
        fc[hub_2_idx[h]] = hubs[h]['fc']
        cap_lim[hub_2_idx[h]] = hubs[h]['cap']

    for i in range(len(ods)):
        hhs = ods[i]['path'][1:-1]
        for hub in hhs:
            cap_row.append(i)
            cap_col.append(hub_2_idx[hub])
            cap_val.append(ods[i]['weight'])

    for i in range(len(ods)):
        demand_row.append(i)
        demand_col.append(od_2_idx[ods[i]['od']])
        demand_val.append(ods[i]['weight'])

    for odp in od:
        demand_lim[od_2_idx[odp]] = od[odp]['weight']
    

    
    m_demand = coo_matrix((demand_val,(demand_row,demand_col)),shape=(len(ods),len(od)))    
    m_cap = coo_matrix((cap_val,(cap_row,cap_col)),shape=(len(ods),len(hubs)))
        
    # dimension check
    # print(len(ods))
    # print(len(od))
    # print(len(hubs))
    
    ########
    #  Model construction and solving
    ########
    
    var = Bool(len(ods))
    var_hub = Bool(len(hubs))
    obj = Minimize(vc.T * var + fc.T * var_hub)
    prob = Problem(obj, constraints=[
        m_demand.T * var == demand_lim,
        m_cap.T * var - mul_elemwise(cap_lim,var_hub) <= 0,
    ])
    
    return prob.solve(), var, var_hub

Scaled expansion test and see computational efficiency

In [158]:
import random
def gen_od_hubs_dist(no_hubs=2, no_nodes=10, no_ods=100):
    nodes = list(range(0,no_nodes))
    
    hubs = {}
    for h in range(no_nodes, no_nodes+ no_hubs):
        hubs[h] = {
            'fc':random.randrange(100000,500000),
            'vc':random.randrange(1,10),
            'cap':random.randrange(5000,30000)
        }
        
    od = {}
    for i in range(no_ods):
        orig = random.choice(nodes)
        dest = random.choice(list(set(nodes) - set([orig])))
        time_allowed = random.randrange(24,96)
        od[orig,dest,time_allowed] = {'weight': random.randrange(500,2000)}
        
    all_nodes = list(nodes) + list(hubs.keys())
    
    dist_matrix = {}
    
    for u in all_nodes:
        for v in all_nodes:
            if u == v:
                continue
            dist_matrix[u,v] = random.randrange(100,4000)
    
    return od,hubs,dist_matrix

In [ ]:
stats = []

In [205]:
%%timeit
od_a,hub_a,dm_a = gen_od_hubs_dist(no_hubs=10)   # 10 hubs, 10 nodes, 100 ods
solve(od=od_a, hubs=hub_a, dm=dm_a)

2312
100
10
2056
99
10
2201
100
10
2351
100
10
1764
100
10
2177
100
10
2620
100
10
2100
97
10
32.1 s ± 3.16 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
